This file reads all of the audio files in a given directory and returns their number and total, mean, max and min length (in seconds). 

In [ ]:
#Imports
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import tensorflow as tf
import numpy as np

import Audio_functions as af
from functools import partial
import datetime

import librosa
import soundfile as sf
import IPython.display as ipd

In [ ]:
# Set some variables
data_folder = 'C:/Users/Toby/Speech_Data/CHiME3/data/audio/16kHz/backgrounds'
#data_folder = 'C:/Users/Toby/Speech_Data/LibriSpeech/train-other-500'
sample_rate = 16384
n_parallel_readers = 4
n_channels = 1

In [ ]:
# Create a data ipeline
data_pipe = tf.data.Dataset.list_files(data_folder + '/*.wav').map(partial(af.read_audio,
                                                                            sample_rate=sample_rate,
                                                                            n_channels=n_channels), 
                                                                    num_parallel_calls=n_parallel_readers)

In [ ]:
sess = tf.Session()
next_file = data_pipe.make_one_shot_iterator().get_next()

In [ ]:
length = 0
file_count = 0
max_file_length = 0
min_file_length = 100000

while True:
    try:
        # Load the next file
        file = sess.run(next_file)
        # Get its length and add it to the total
        len_file = len(file) / sample_rate
        length += len_file
        # Check if this is the longest or shortest file encountered so far. If so, update records.
        if len_file > max_file_length:
            max_file_length = len_file
        if len_file < min_file_length:
            min_file_length = len_file
        file_count += 1
        if file_count % 10 == 0:
            print('{fc} files processed'.format(fc=file_count))
    except tf.errors.OutOfRangeError:
        # When the dataset is exhausted, return the results.
        mean_file_length = length / file_count
        print('Files in folder: {fc}'.format(fc=file_count))
        print('Total length of audio in folder = {l}s ({lh} hours)'.format(l=round(length, 2), 
                                                                           lh=str(datetime.timedelta(seconds=length))))
        print('Maximum file length = {maxfl}s'.format(maxfl=round(max_file_length, 2)))
        print('Minimum file length = {minfl}s'.format(minfl=round(min_file_length, 2)))
        print('Mean file length = {meanfl}s'.format(meanfl=round(mean_file_length, 2)))
        break